In [1]:
from IPython.display import HTML, display
css_file_path = "../../design/custom.css"
styles = "<link rel='stylesheet' href='%s'>" % css_file_path
display(HTML(styles))

# 3.3 $\quad$ CVXPY

Para la resolución de problemas de optimización convexa en python existen básicamente 3 métodos:

- Implementación de los distintos algoritmos de optimización (solvers), como, por ejemplo, Gurovi, Scipy PuLP, GLPK, CPLEX, etc.

- Diseño de tu propio algoritmo (modelos de Machine Learning).

- Usar una librería de alto nivel (CVXPY), que integra los solvers estandar usando un lenguaje de alto nivel.

CVXPY es un lenguaje específico para la optimización convexa integrado en Python. Sus características principales incluyen:
  
- **Expresión Natural de Problemas**: 
  - Permite a los usuarios formular problemas de optimización convexa utilizando una sintaxis de alto nivel que sigue la lógica matemática.

- **Compatibilidad con Solucionadores Conocidos**: 
  - Facilita el uso de los solvers de optimización más conocidos
  
- **Verifica que el problema es convexo**:
  - La librería tiene herramientas para comprobar si el problema de optimización es convexo.

Puedes encontrar toda la documentación sobre la librería en [CVXPY](https://www.cvxpy.org/index.html)

Presentación de [Convex Optimization with CVXPY](https://www.youtube.com/watch?v=kXqu-TqEl7Q)

### Discipline Convex Programming

#### Concepto y Fundamentos
- **Definición**: Disciplined Convex Programming (DCP) es un sistema de reglas y técnicas para la construcción de problemas de optimización convexa.
- **Objetivo**: Asegurar la convexidad de los problemas de optimización formulados, facilitando la búsqueda de soluciones globales óptimas.

#### Principios Clave
1. **Composición de Funciones Convexas**: Las funciones en un problema DCP deben combinarse de acuerdo con ciertas reglas que preservan la convexidad.
2. **Reglas de Curvatura**: Identifican si una expresión es convexa, cóncava o afín, basándose en la operación y las funciones involucradas.
3. **Restricciones DCP**: Solo ciertas formas de restricciones son permitidas para mantener la convexidad del problema.

#### Aplicación en CVXPY
- **CVXPY y DCP**: CVXPY utiliza DCP para verificar automáticamente la convexidad de un problema formulado.
- **Beneficios**:
  - Simplifica la formulación de problemas convexos.
  - Evita errores comunes al garantizar que el problema es convexo y, por lo tanto, solucionable.


### Componentes de un Problema de Optimización usando CVXPY

Recuerda lo que hemos visto en el notebook anterior:

Nuestro objetivo es **mimimizar/maximizar $f_0(x| \theta)$** sujeto a $f_i(x| \theta) \leq b_i$,   $\qquad i= 1,...,m$

donde:

- $x = (x_1, x_2,..., x_n)$ son las variables

- $\theta = (\theta_1, \theta_2,..., \theta_n)$ son los parámetros

- $f_0: \mathbb{R}^n \rightarrow \mathbb{R}$  es la función objetivo

- $f_i: \mathbb{R}^n \rightarrow \mathbb{R}$ son las restricciones

In [2]:
import numpy as np
import pandas as pd
import cvxpy as cp

#### Variables de decisión (parámetros entrenables)
- **Definición**: Representan las cantidades que queremos optimizar. Representadas por el vector $w$.
- **Ejemplo**: En un problema de asignación de recursos, las variables podrían ser la cantidad de recursos asignados a diferentes tareas.


Para recordarlo, puedes entender el concepto de `variable` como algo que se va a modificar (va a variar) durante el ajuste del optimizador.

In [3]:
# Una variable entrenable de 2 componentes (2 ponderaciones)
w = cp.Variable(2) # (w1, w2)
w

Variable((2,), var1)

In [4]:
# Es equivalente a definir directamente un vector de variables (5 ponderaciones)
w = cp.Variable((5,)) # (w1, w2, w3, w4, w5)
w

Variable((5,), var2)

In [5]:
# También es posible definir una matriz de variables
m = cp.Variable((5,2))
m
# [(m11, m12), 
#  (m21, m22), 
#  (m31, m32), 
#  (m41, m42), 
#  (m51, m52)]

Variable((5, 2), var3)

#### Función Objetivo
- **Definición**: Es la función que queremos minimizar o maximizar. Representada por $f_0$.
- **Ejemplo**: En un problema de minimización de costos, la función objetivo podría ser el costo total en función de las variables de decisión.


In [6]:
# Minimizamos la suma de las componentes del vector v
cp.Minimize(cp.sum(w))

Minimize(Expression(AFFINE, UNKNOWN, ()))

In [7]:
# Minimizamos la norma L1 (suma de valores absolutos) del vector a
cp.Minimize(cp.norm(w, 1))

Minimize(Expression(CONVEX, NONNEGATIVE, ()))

In [8]:
# Maximizamos la suma de los logaritmos de las componentes del vector v
cp.Maximize(cp.sum(cp.log(w)))

Maximize(Expression(CONCAVE, UNKNOWN, ()))

#### Restricciones
- **Definición**: Son las condiciones que las variables de decisión deben cumplir. Representadas por $f_i$.
- **Ejemplo**: En un problema de minimización de costos, podría ser los costes mínimos necesarios para cada tarea.

- Las restricciones se modelan con expresiones de igualdad y desigualdad
con ==, >=, <=. 

- Las desigualdades estrictas < y > no están permitidas. $x<5$ se sustituye por $x+\epsilon \leq 5$ siendo $\epsilon$ un número muy pequeño

- Las expresiones de desigualdad se interpretan elemento a elemento y sigiuendo
las reglas de interpretación para escalares, vectores y matrices al estilo 
de numpy (broadcasting).

In [9]:
# los 5 elementos del vector v debe ser mayor que 6
w <= 6

Inequality(Variable((5,), var2))

In [10]:
# Cada elemento de v debe ser mayor que cada elemento del array c
c = np.array([1, 3, 5, 10, 2])
w >= c 

Inequality(Constant(CONSTANT, NONNEGATIVE, (5,)))

In [11]:
# Las expresiones las podemos asignar a variables o agruparlas en listas
restriccion_1 = (m <= 10)
restriccion_2 = (m >= 0)
constraints = [restriccion_1, restriccion_2]
constraints.append(w<=c)
constraints

[Inequality(Variable((5, 2), var3)),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Variable((5,), var2))]

Si pensamos en que CVXPY está construido sobre numpy, podemos imaginar que es posible definir restricciones sobre subconjuntos de variables usando el indexado o el slicing:

In [12]:
# La variable m31 debe ser <= 9
m[3, 1] <= 9

Inequality(Expression(AFFINE, UNKNOWN, ()))

In [13]:
# Las variables de la primera columna de las 4 primeras filas deben ser <= 8
m[:4, 0] <= 8

Inequality(Expression(AFFINE, UNKNOWN, (4,)))

In [14]:
# Podemos mezclar nuestro código con la construcción
# de restricciones a nuestra conveniencia

other_constraints = []
for i in range(w.shape[0]):
    other_constraints.append(w[i] >= i - 2)

other_constraints

[Inequality(Constant(CONSTANT, NONPOSITIVE, ())),
 Inequality(Constant(CONSTANT, NONPOSITIVE, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ()))]

### Operadores y Funciones
La librería trata los operadores +, -, *, / y @ como funciones, conservando la semántica de numpy.

In [15]:
w*2

Expression(AFFINE, UNKNOWN, (5,))

In [16]:
w + w*2

Expression(AFFINE, UNKNOWN, (5,))

In [17]:
# w tiene shape (5,)
# m tiene shape (5,2)
w @ m # tiene shape (2,)

Expression(UNKNOWN, UNKNOWN, (2,))

In [18]:
# m tiene shape (5,2)
# m.T tiene shape (2,5)
m @ m.T

Expression(UNKNOWN, UNKNOWN, (5, 5))

OJO: Algunas funciones se aplican a cada elemento, como en numpy:

In [19]:
# valor absoluto de cada elemento del vector
cp.abs(w)

Expression(CONVEX, NONNEGATIVE, (5,))

In [20]:
# para cada elemento de la matriz se calcula e^{m_ij}
cp.exp(m)

Expression(CONVEX, NONNEGATIVE, (5, 2))

La librería contiene un conjunto diverso de funciones para realizar
la mayoría de los cálculos matemáticos.

-  Consultar una lista mas completa en la [documentación](https://www.cvxpy.org/tutorial/functions/index.html)